# Data ingestion and formatting

This notebook explains how to convert the Climate TRACE dataset to a format that is more appropriate for data science. 

```{note}
This section is relevant for data engineers, or data scientists who want to understand how the data 
has been prepared. Skip if you just want to access the final, prepared data.
```

The original data from Climate TRACE is offered as a series of CSV files bundled in ZIP archives. That format is universally understood, but it is not the most effective for effective analysis.

Instead, we are going to use the Parquet format. This format has a number of advantages:
- it is _column-based_ : data systems can process big chunks of data at once, rather than line by line. Also, depending on the information requested, systems will read only the relevant columns and skip the rest very effectively
- it is _structured_ : basic information about numbers, categories, ... are preserved. This provides a large speed boost
- it is _universal_ : most modern data systems will be able to read it

```{admonition} TODO
complete this notebook and publish the source code.
```

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import logging
logging.basicConfig(level=logging.DEBUG)


In [3]:
import polars as pl
import plotly.express as px

from ctrace.constants import *
import ctrace as ct

In [5]:
ldf = ct.data.load_source_compact()

DEBUG:ctrace.data:Fetching path agriculture.zip
DEBUG:ctrace.data:Opening path agriculture.zip from /home/tjhunter/.cache/climate_trace/v2-2023/agriculture.zip
DEBUG:ctrace.data:sources: ['enteric-fermentation-cattle-feedlot_emissions-sources.csv', 'manure-management-cattle-feedlot_emissions-sources.csv', 'rice-cultivation_emissions-sources.csv', 'synthetic-fertilizer-application_emissions-sources.csv', 'enteric-fermentation-cattle-pasture_emissions-sources.csv', 'cropland-fires_emissions-sources.csv', 'manure-left-on-pasture-cattle_emissions-sources.csv']
DEBUG:ctrace.data:opening agriculture.zip / enteric-fermentation-cattle-feedlot_emissions-sources.csv
DEBUG:ctrace.data:opening agriculture.zip / enteric-fermentation-cattle-feedlot_emissions-sources_confidence.csv
DEBUG:ctrace.data:wrote /tmp/enteric-fermentation-cattle-feedlot_emissions-sources.parquet
DEBUG:ctrace.data:opening agriculture.zip / manure-management-cattle-feedlot_emissions-sources.csv
DEBUG:ctrace.data:opening agricu

In [6]:
ldf

<LazyFrame [60 cols, {"source_id": UInt64 … "ct_file": Enum(categories=['aluminum', 'bauxite-mining', 'biological-treatment-of-solid-waste-and-biogenic', 'cement', 'chemicals', 'coal-mining', 'copper-mining', 'cropland-fires', 'domestic-aviation', 'domestic-shipping', 'electricity-generation', 'enteric-fermentation-cattle-feedlot', 'enteric-fermentation-cattle-pasture', 'enteric-fermentation-other', 'fluorinated-gases', 'forest-land-clearing', 'forest-land-degradation', 'forest-land-fires', 'incineration-and-open-burning-of-waste', 'international-aviation', 'international-shipping', 'iron-mining', 'manure-left-on-pasture-cattle', 'manure-management-cattle-feedlot', 'manure-management-other', 'net-forest-land', 'net-shrubgrass', 'net-wetland', 'oil-and-gas-production-and-transport', 'oil-and-gas-refining', 'other-agricultural-soil-emissions', 'other-energy-use', 'other-fossil-fuel-operations', 'other-manufacturing', 'other-onsite-fuel-usage', 'other-transport', 'petrochemicals', 'pulp-and-paper', 'railways', 'removals', 'residential-and-commercial-onsite-fuel-usage', 'rice-cultivation', 'road-transportation', 'rock-quarrying', 'sand-quarrying', 'shrubgrass-fires', 'solid-fuel-transformation', 'solid-waste-disposal', 'steel', 'synthetic-fertilizer-application', 'wastewater-treatment-and-discharge', 'water-reservoirs', 'wetland-fires'])}] at 0x78AA52304790>

In [4]:
df = ct.read_country_emissions()

DEBUG:ctrace.data:Fetching path agriculture.zip
DEBUG:ctrace.data:Opening path agriculture.zip from /home/tjhunter/.cache/climate_trace/v2-2023/agriculture.zip
DEBUG:ctrace.data:sources: ['other-agricultural-soil-emissions_country_emissions.csv', 'enteric-fermentation-other_country_emissions.csv', 'manure-management-other_country_emissions.csv', 'rice-cultivation_country_emissions.csv', 'enteric-fermentation-cattle-feedlot_country_emissions.csv', 'manure-management-cattle-feedlot_country_emissions.csv', 'synthetic-fertilizer-application_country_emissions.csv', 'enteric-fermentation-cattle-pasture_country_emissions.csv', 'cropland-fires_country_emissions.csv', 'manure-left-on-pasture-cattle_country_emissions.csv']
DEBUG:ctrace.data:opening agriculture.zip / other-agricultural-soil-emissions_country_emissions.csv
DEBUG:ctrace.data:opening agriculture.zip / enteric-fermentation-other_country_emissions.csv
DEBUG:ctrace.data:opening agriculture.zip / manure-management-other_country_emission

In [7]:
pl.read_parquet("/tmp/cement_emissions-sources.parquet")

source_id,iso3_country,original_inventory_sector,start_time,end_time,temporal_granularity,gas,emissions_quantity,emissions_factor,emissions_factor_units,capacity,capacity_units,capacity_factor,activity,activity_units,created_date,modified_date,source_name,source_type,lat,lon,other1,other2,other3,other4,other5,other6,other7,other8,other9,other10,other11,other12,other1_def,other2_def,other3_def,other4_def,other5_def,other6_def,other7_def,other8_def,other9_def,other10_def,other11_def,other12_def,geometry_ref,conf_source_type,conf_capacity,conf_capacity_factor,conf_activity,conf_co2_emissions_factor,conf_ch4_emissions_factor,conf_n2o_emissions_factor,conf_co2_emissions,conf_ch4_emissions,conf_n2o_emissions,conf_total_co2e_20yrgwp,conf_total_co2e_100yrgwp,ct_package,ct_file
u64,enum,enum,datetime[μs],datetime[μs],enum,enum,f64,f64,str,f64,str,f64,f64,str,datetime[μs],datetime[μs],str,str,f64,f64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum
18655090,"""AFG""","""cement""",2016-02-01 00:00:00,2016-02-29 00:00:00,"""month""","""co2e_100yr""",8508.0,null,null,360000.0,"""T of cement""",0.27,8000.0,"""T of cement""",2023-10-12 00:00:00,2024-02-27 14:36:31.515312,"""Pol-e Khomri c…","""Wet""",35.9658,68.686338,"""1.11725754""","""8938.0""","""0.52""","""4160.0""","""0.52552102""","""4204.0""",null,null,null,null,null,null,"""Direct and Ind…","""Direct and Ind…","""Calcination em…","""Calcination em…","""Fuel emissions…","""Fuel emissions…",null,null,null,null,null,null,"""trace_68.68633…","""very high""","""high""","""medium""","""medium""","""medium""","""medium""","""medium""","""medium""","""medium""","""medium""","""medium""","""medium""","""manufacturing""","""cement"""
18655090,"""AFG""","""cement""",2021-11-01 00:00:00,2021-11-30 00:00:00,"""month""","""co2e_100yr""",2000.0,null,null,360000.0,"""T of cement""",0.07,2000.0,"""T of cement""",2023-10-12 00:00:00,null,"""Pol-e Khomri c…","""Wet""",35.9658,68.686338,"""1.0564037""","""2113.0""","""0.52""","""1040.0""","""0.47003662""","""940.0""",null,null,null,null,null,null,"""Direct and Ind…","""Direct and Ind…","""Calcination em…","""Calcination em…","""Fuel emissions…","""Fuel emissions…",null,null,null,null,null,null,"""trace_68.68633…","""very high""","""high""","""medium""","""medium""","""medium""","""medium""","""medium""","""medium""","""medium""","""medium""","""medium""","""medium""","""manufacturing""","""cement"""
18655090,"""AFG""","""cement""",2020-10-01 00:00:00,2020-10-31 00:00:00,"""month""","""co2e_100yr""",2000.0,null,null,360000.0,"""T of cement""",0.07,2000.0,"""T of cement""",2023-10-12 00:00:00,null,"""Pol-e Khomri c…","""Wet""",35.9658,68.686338,"""1.0564037""","""2113.0""","""0.52""","""1040.0""","""0.47003662""","""940.0""",null,null,null,null,null,null,"""Direct and Ind…","""Direct and Ind…","""Calcination em…","""Calcination em…","""Fuel emissions…","""Fuel emissions…",null,null,null,null,null,null,"""trace_68.68633…","""very high""","""high""","""medium""","""medium""","""medium""","""medium""","""medium""","""medium""","""medium""","""medium""","""medium""","""medium""","""manufacturing""","""cement"""
18655090,"""AFG""","""cement""",2016-12-01 00:00:00,2016-12-31 00:00:00,"""month""","""ch4""",null,null,"""field_not_mode…",360000.0,"""T of cement""",0.27,8000.0,"""T of cement""",2023-10-12 00:00:00,null,"""Pol-e Khomri c…","""Wet""",35.9658,68.686338,"""1.11725754""","""8938.0""","""0.52""","""4160.0""","""0.52552102""","""4204.0""",null,null,null,null,null,null,"""Direct and Ind…","""Direct and Ind…","""Calcination em…","""Calcination em…","""Fuel emissions…","""Fuel emissions…",null,null,null,null,null,null,"""trace_68.68633…","""very high""","""high""","""medium""","""medium""","""medium""","""medium""","""medium""","""medium""","""medium""","""medium""","""medium""","""medium""","""manufacturing""","""cement"""
18655090,"""AF

In [18]:
ldf.head(3).collect()

source_id,iso3_country,original_inventory_sector,start_time,end_time,temporal_granularity,gas,emissions_quantity,emissions_factor,emissions_factor_units,capacity,capacity_units,capacity_factor,activity,activity_units,created_date,modified_date,source_name,source_type,lat,lon,other1,other2,other3,other4,other5,other6,other7,other8,other9,other10,other11,other12,other1_def,other2_def,other3_def,other4_def,other5_def,other6_def,other7_def,other8_def,other9_def,other10_def,other11_def,other12_def,geometry_ref,conf_source_type,conf_capacity,conf_capacity_factor,conf_activity,conf_co2_emissions_factor,conf_ch4_emissions_factor,conf_n2o_emissions_factor,conf_co2_emissions,conf_ch4_emissions,conf_n2o_emissions,conf_total_co2e_20yrgwp,conf_total_co2e_100yrgwp,ct_package,ct_file
u64,enum,enum,datetime[μs],datetime[μs],enum,enum,f64,f64,str,f64,str,f64,f64,str,datetime[μs],datetime[μs],str,str,f64,f64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum
3154896,"""CHN""","""enteric-fermen…",2015-01-01 00:00:00,2015-12-31 00:00:00,"""annual""","""n2o""",0.0,0.0,"""T of N2O per a…",0.0,"""hectares """,0.0,0.0,"""animals""",2023-08-31 00:00:00,null,"""CHN_beef_1""","""beef""",37.772359,118.60246,"""2016""","""modeled""",null,null,null,null,null,null,null,null,null,null,"""year active""","""cattle_populat…","""potential_tota…","""dairy_or_beef_…","""dairy_total_ot…","""other_cattle_e…",null,null,null,null,null,null,"""trace_118.6024…",null,null,null,null,null,null,null,null,null,null,null,null,"""agriculture""","""enteric-fermen…"
3154896,"""CHN""","""enteric-fermen…",2015-01-01 00:00:00,2015-12-31 00:00:00,"""annual""","""co2e_100yr""",0.0,0.0,null,0.0,"""hectares """,0.0,0.0,"""animals""",2023-08-31 00:00:00,null,"""CHN_beef_1""","""beef""",37.772359,118.60246,"""2016""","""modeled""",null,null,null,null,null,null,null,null,null,null,"""year active""","""cattle_populat…","""potential_tota…","""dairy_or_beef_…","""dairy_total_ot…","""other_cattle_e…",null,null,null,null,null,null,"""trace_118.6024…",null,null,null,null,null,null,null,null,null,null,null,null,"""agriculture""","""enteric-fermen…"
3154896,"""CHN""","""enteric-fermen…",2015-01-01 00:00:00,2015-12-31 00:00:00,"""annual""","""co2e_20yr""",0.0,0.0,null,0.0,"""hectares """,0.0,0.0,"""animals""",2023-08-31 00:00:00,null,"""CHN_beef_1""","""beef""",37.772359,118.60246,"""2016""","""modeled""",null,null,null,null,null,null,null,null,null,null,"""year active""","""cattle_populat…","""potential_tota…","""dairy_or_beef_…","""dairy_total_ot…","""other_cattle_e…",null,null,null,null,null,null,"""trace_118.6024…",null,null,null,null,null,null,null,null,null,null,null,null,"""agriculture""","""enteric-fermen…"


In [8]:
sorted(df[ORIGINAL_INVENTORY_SECTOR].value_counts()[ORIGINAL_INVENTORY_SECTOR].to_list())

['aluminum',
 'bauxite-mining',
 'biological-treatment-of-solid-waste-and-biogenic',
 'cement',
 'chemicals',
 'coal-mining',
 'copper-mining',
 'cropland-fires',
 'domestic-aviation',
 'domestic-shipping',
 'electricity-generation',
 'enteric-fermentation-cattle-feedlot',
 'enteric-fermentation-cattle-pasture',
 'enteric-fermentation-other',
 'fluorinated-gases',
 'forest-land-clearing',
 'forest-land-degradation',
 'forest-land-fires',
 'incineration-and-open-burning-of-waste',
 'international-aviation',
 'international-shipping',
 'iron-mining',
 'manure-left-on-pasture-cattle',
 'manure-management-cattle-feedlot',
 'manure-management-other',
 'net-forest-land',
 'net-shrubgrass',
 'net-wetland',
 'oil-and-gas-production-and-transport',
 'oil-and-gas-refining',
 'other-agricultural-soil-emissions',
 'other-energy-use',
 'other-fossil-fuel-operations',
 'other-manufacturing',
 'other-onsite-fuel-usage',
 'other-transport',
 'petrochemicals',
 'pulp-and-paper',
 'railways',
 'removals

In [9]:
sorted(df[EMISSIONS_QUANTITY_UNITS].value_counts()[EMISSIONS_QUANTITY_UNITS].to_list())

['tonnes']

In [10]:
sorted(df[CT_PACKAGE].value_counts()[CT_PACKAGE].to_list())

['agriculture',
 'buildings',
 'fluorinated_gases',
 'forestry_and_land_use',
 'fossil_fuel_operations',
 'manufacturing',
 'mineral_extraction',
 'power',
 'transportation',
 'waste']

In [11]:
sorted(df[CT_FILE].value_counts()[CT_FILE].to_list())

['aluminum',
 'bauxite-mining',
 'biological-treatment-of-solid-waste-and-biogenic',
 'cement',
 'chemicals',
 'coal-mining',
 'copper-mining',
 'cropland-fires',
 'domestic-aviation',
 'domestic-shipping',
 'electricity-generation',
 'enteric-fermentation-cattle-feedlot',
 'enteric-fermentation-cattle-pasture',
 'enteric-fermentation-other',
 'fluorinated-gases',
 'forest-land-clearing',
 'forest-land-degradation',
 'forest-land-fires',
 'incineration-and-open-burning-of-waste',
 'international-aviation',
 'international-shipping',
 'iron-mining',
 'manure-left-on-pasture-cattle',
 'manure-management-cattle-feedlot',
 'manure-management-other',
 'net-forest-land',
 'net-shrubgrass',
 'net-wetland',
 'oil-and-gas-production-and-transport',
 'oil-and-gas-refining',
 'other-agricultural-soil-emissions',
 'other-energy-use',
 'other-fossil-fuel-operations',
 'other-manufacturing',
 'other-onsite-fuel-usage',
 'other-transport',
 'petrochemicals',
 'pulp-and-paper',
 'railways',
 'removals

In [12]:
SUBSECTORS = [
    "aluminum",
    "bauxite-mining",
    "biological-treatment-of-solid-waste-and-biogenic",
    "cement",
    "chemicals",
    "coal-mining",
    "copper-mining",
    "cropland-fires",
    "domestic-aviation",
    "domestic-shipping",
    "electricity-generation",
    "enteric-fermentation-cattle-feedlot",
    "enteric-fermentation-cattle-pasture",
    "enteric-fermentation-other",
    "fluorinated-gases",
    "forest-land-clearing",
    "forest-land-degradation",
    "forest-land-fires",
    "incineration-and-open-burning-of-waste",
    "international-aviation",
    "international-shipping",
    "iron-mining",
    "manure-left-on-pasture-cattle",
    "manure-management-cattle-feedlot",
    "manure-management-other",
    "net-forest-land",
    "net-shrubgrass",
    "net-wetland",
    "oil-and-gas-production-and-transport",
    "oil-and-gas-refining",
    "other-agricultural-soil-emissions",
    "other-energy-use",
    "other-fossil-fuel-operations",
    "other-manufacturing",
    "other-onsite-fuel-usage",
    "other-transport",
    "petrochemicals",
    "pulp-and-paper",
    "railways",
    "removals",
    "residential-and-commercial-onsite-fuel-usage",
    "rice-cultivation",
    "road-transportation",
    "rock-quarrying",
    "sand-quarrying",
    "shrubgrass-fires",
    "solid-fuel-transformation",
    "solid-waste-disposal",
    "steel",
    "synthetic-fertilizer-application",
    "wastewater-treatment-and-discharge",
    "water-reservoirs",
    "wetland-fires",
]


def _code_subsector():
    for s in SUBSECTORS:
        s2 = s.upper().replace("-", "_")
        print(f"{s2} = '{s}'")

    print(f"\n\n")
    sub = ", ".join([f"'{s}'" for s in SUBSECTORS])
    print(f"SubSector = Literal[{sub}]")
    print(f"\n\n")



In [13]:
_code_subsector()

ALUMINUM = 'aluminum'
BAUXITE_MINING = 'bauxite-mining'
BIOLOGICAL_TREATMENT_OF_SOLID_WASTE_AND_BIOGENIC = 'biological-treatment-of-solid-waste-and-biogenic'
CEMENT = 'cement'
CHEMICALS = 'chemicals'
COAL_MINING = 'coal-mining'
COPPER_MINING = 'copper-mining'
CROPLAND_FIRES = 'cropland-fires'
DOMESTIC_AVIATION = 'domestic-aviation'
DOMESTIC_SHIPPING = 'domestic-shipping'
ELECTRICITY_GENERATION = 'electricity-generation'
ENTERIC_FERMENTATION_CATTLE_FEEDLOT = 'enteric-fermentation-cattle-feedlot'
ENTERIC_FERMENTATION_CATTLE_PASTURE = 'enteric-fermentation-cattle-pasture'
ENTERIC_FERMENTATION_OTHER = 'enteric-fermentation-other'
FLUORINATED_GASES = 'fluorinated-gases'
FOREST_LAND_CLEARING = 'forest-land-clearing'
FOREST_LAND_DEGRADATION = 'forest-land-degradation'
FOREST_LAND_FIRES = 'forest-land-fires'
INCINERATION_AND_OPEN_BURNING_OF_WASTE = 'incineration-and-open-burning-of-waste'
INTERNATIONAL_AVIATION = 'international-aviation'
INTERNATIONAL_SHIPPING = 'international-shipping'
IRON_M

In [14]:
SECTORS = [
    "agriculture",
    "buildings",
    "fluorinated_gases",
    "forestry_and_land_use",
    "fossil_fuel_operations",
    "manufacturing",
    "mineral_extraction",
    "power",
    "transportation",
    "waste",
]


def _code_sector():
    for s in SECTORS:
        s2 = s.upper().replace("-", "_")
        print(f"{s2} = '{s}'")

    print(f"\n\n")
    sub = ", ".join([f"'{s}'" for s in SECTORS])
    print(f"SubSector = Literal[{sub}]")
    print(f"\n\n")

In [15]:
_code_sector()

AGRICULTURE = 'agriculture'
BUILDINGS = 'buildings'
FLUORINATED_GASES = 'fluorinated_gases'
FORESTRY_AND_LAND_USE = 'forestry_and_land_use'
FOSSIL_FUEL_OPERATIONS = 'fossil_fuel_operations'
MANUFACTURING = 'manufacturing'
MINERAL_EXTRACTION = 'mineral_extraction'
POWER = 'power'
TRANSPORTATION = 'transportation'
WASTE = 'waste'



SubSector = Literal['agriculture', 'buildings', 'fluorinated_gases', 'forestry_and_land_use', 'fossil_fuel_operations', 'manufacturing', 'mineral_extraction', 'power', 'transportation', 'waste']





In [16]:

ORIGINAL_INVENTORY_SECTORS = [
    "aluminum",
    "bauxite-mining",
    "biological-treatment-of-solid-waste-and-biogenic",
    "cement",
    "chemicals",
    "coal-mining",
    "copper-mining",
    "cropland-fires",
    "domestic-aviation",
    "domestic-shipping",
    "electricity-generation",
    "enteric-fermentation-cattle-feedlot",
    "enteric-fermentation-cattle-pasture",
    "enteric-fermentation-other",
    "fluorinated-gases",
    "forest-land-clearing",
    "forest-land-degradation",
    "forest-land-fires",
    "incineration-and-open-burning-of-waste",
    "international-aviation",
    "international-shipping",
    "iron-mining",
    "manure-left-on-pasture-cattle",
    "manure-management-cattle-feedlot",
    "manure-management-other",
    "net-forest-land",
    "net-shrubgrass",
    "net-wetland",
    "oil-and-gas-production-and-transport",
    "oil-and-gas-refining",
    "other-agricultural-soil-emissions",
    "other-energy-use",
    "other-fossil-fuel-operations",
    "other-manufacturing",
    "other-onsite-fuel-usage",
    "other-transport",
    "petrochemicals",
    "pulp-and-paper",
    "railways",
    "removals",
    "residential-and-commercial-onsite-fuel-usage",
    "rice-cultivation",
    "road-transportation",
    "rock-quarrying",
    "sand-quarrying",
    "shrubgrass-fires",
    "solid-fuel-transformation",
    "solid-waste-disposal",
    "steel",
    "synthetic-fertilizer-application",
    "wastewater-treatment-and-discharge",
    "water-reservoirs",
    "wetland-fires",
]

def _code_original_inventory_sector():
    for s in ORIGINAL_INVENTORY_SECTORS:
        s2 = s.upper().replace("-", "_")
        print(f"{s2} = '{s}'")

    print(f"\n\n")
    sub = ", ".join([f"'{s}'" for s in ORIGINAL_INVENTORY_SECTORS])
    print(f"Sector = Literal[{sub}]")
    print(f"\n\n")

_code_original_inventory_sector()

ALUMINUM = 'aluminum'
BAUXITE_MINING = 'bauxite-mining'
BIOLOGICAL_TREATMENT_OF_SOLID_WASTE_AND_BIOGENIC = 'biological-treatment-of-solid-waste-and-biogenic'
CEMENT = 'cement'
CHEMICALS = 'chemicals'
COAL_MINING = 'coal-mining'
COPPER_MINING = 'copper-mining'
CROPLAND_FIRES = 'cropland-fires'
DOMESTIC_AVIATION = 'domestic-aviation'
DOMESTIC_SHIPPING = 'domestic-shipping'
ELECTRICITY_GENERATION = 'electricity-generation'
ENTERIC_FERMENTATION_CATTLE_FEEDLOT = 'enteric-fermentation-cattle-feedlot'
ENTERIC_FERMENTATION_CATTLE_PASTURE = 'enteric-fermentation-cattle-pasture'
ENTERIC_FERMENTATION_OTHER = 'enteric-fermentation-other'
FLUORINATED_GASES = 'fluorinated-gases'
FOREST_LAND_CLEARING = 'forest-land-clearing'
FOREST_LAND_DEGRADATION = 'forest-land-degradation'
FOREST_LAND_FIRES = 'forest-land-fires'
INCINERATION_AND_OPEN_BURNING_OF_WASTE = 'incineration-and-open-burning-of-waste'
INTERNATIONAL_AVIATION = 'international-aviation'
INTERNATIONAL_SHIPPING = 'international-shipping'
IRON_M